# 学習に関するテクニック
## パラメータの更新(最適化手法，Optimizer)
- SGD
- Momentum(前の移動方向を考慮)
- AdaGrad(パラメータごとに更新量を考慮して学習係数を減衰)
- Adam(Momentum + AdaGrad)

In [ ]:
import numpy as np

###SGD###
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads

###Momentum###
class Momentum:
    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    def update(self, params, grads):
        if self.v is None:
            self.v={}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)    #v=0のとき（初回更新時）に,「W」と同じ形の「v」を作成
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

class AdaGrad:
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key]/(np.sqrt(self.h[key])+1e-7)

class Adam:

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)



## 重みの初期値
- Xavierの初期値(標準偏差　1/sqrt(n)　で重みを初期化する．nは前の層のノード数)．活性化関数がsigmoidやtanh関数(０付近が線形関数とみなせるもの)の時は適している
- Heの初期値(標準偏差　sqrt(2/n) で重みを初期化する．nは前の層のノード数)．活性化関数がReLUのとき適している

## Batch Noremalization
- 各層のアクティベーション(活性化関数の後の出力データ)に強制的に広がりを持たせる
- 学習を速く進行させられる（lrを大きくできる），初期値にそれほど依存しない，過学習を抑制する（Dropoutなどの必要性を減らす）
- 具体的には，ミニバッチごとに平均0分散1になるように正規化を行う
$$
    x_i ←　\frac{x_i - \mu}{\sqrt{\sigma^2 + 1e-7}}
$$

## 正則化
過学習(over fitting)に対処する
- wight decay(荷重減衰)
- Dropout．よく用いられる．訓練時に学習させるユニットを限定する（ランダムに選ぶ）．

In [1]:
class Dropout:
    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None
    def forward(self, x, train_flg = True):
        if train_flg:
            self.mask= np.random.rand(*x.shape) > self.dropout_ratio
            return x*self.mask
        else:
            return x*(1-self.dropout_ratio)
    def backward(self, dout):
        return dout * self.mask

## ハイパーパラメータの検証
検証データ(validation data)を用いてハイパーパラメータを検証する